In [146]:
%serialconnect

Connecting to Serial /dev/ttyUSB4 baud=115200 
Ready.


In [11]:
# Communications according to:
# https://arduino.stackexchange.com/questions/23684/stpm10-using-arduino-spi-communication/23742

# spi outline here: https://forum.micropython.org/viewtopic.php?f=16&t=1705&sid=a941eec9bce447f4046c2bd164953481&start=10

# teardowns of these devices here: https://www.youtube.com/watch?v=MeTVCbjFft4

In [152]:
%sendtofile main.py

import machine

SPISCS = machine.Pin(27, machine.Pin.OUT)
SPISDA = machine.Pin(25)  # 12 is bad causes total crash
SPISCL = machine.Pin(14, machine.Pin.OUT)
SPISYN = machine.Pin(26, machine.Pin.OUT)
SPISCS.value(1)
SPISYN.value(1)
spi = machine.SPI(miso=SPISDA, mosi=machine.Pin(13, machine.Pin.OUT), sck=SPISCL, 
                  baudrate=32000000, firstbit=machine.SPI.MSB)


Sent 10 lines (377 bytes) to main.py.


In [108]:
spi.init()

In [107]:
help(spi)

object SoftSPI(baudrate=1200000, polarity=0, phase=0, sck=14, mosi=13, miso=25) is of type SoftSPI
  init -- <function>
  deinit -- <function>
  read -- <function>
  readinto -- <function>
  write -- <function>
  write_readinto -- <function>
  MSB -- 0
  LSB -- 1


In [153]:
%sendtofile --append main.py

import time
def write_reg(addr, val):
    val = (val << 7) | (addr << 1);  # 6-bit addresses!
    spi.deinit()
    SPISDA.init(machine.Pin.OUT)
    SPISYN.value(1)
    SPISCL.value(1)
    time.sleep_us(10)
    SPISCS.value(0)
    SPISYN.value(0)
    for i in range(8):
        time.sleep_us(10);
        SPISCL.value(0)
        SPISDA.value(1 if (val & 0x80) else 0);
        time.sleep_us(10);
        SPISCL.value(1)
        val <<= 1;

    SPISYN.value(1)
    SPISCS.value(1)
    SPISDA.init(machine.Pin.IN, machine.Pin.PULL_UP)
    spi.init()

def read_regs(n):
    SPISYN.value(0)
    SPISCS.value(0)
    SPISYN.value(1)
    res = [ ]
    for i in range(n):
        res.append(spi.read(4))
    SPISCS.value(1)
    return res

Sent 32 lines (729 bytes) to main.py.


In [ ]:
import ustruct
write_reg(47, 1) #;  // set config bit 47
x = read_regs(8) #;  // read all reg data into array
print(" ".join([hex(ustruct.unpack("I", y)[0])[-8:]  for y in x]))


[missing-OK]_e x(0i 00 
8000001300000007000500814070f9f000080f030
ffef88e1007000b807f8f0f808b078fff088e000000080000000fe0f01000000b70
000800a0800000
070000707f7e08e0008
fe9fff80xcfb80fff0a00800875b480070d00c040
08e000770ff8e40000f00804b0ff0f000f90fe0000008bb8ef07007f400d7070000e1x50800070f70000801000
ff00007406070f9ae8080fa80b0af980b70707e08f080e0040
eff08f0000000002f080700000fa0807
 my 870fef6805b00004800e  tcl>
[missing-OK]ine 52, in <listcomp>>

>
..
[missing-OK]ost>
>>
[missing-OK]:] ox
[missing-OK]fe0 f f00..
[missing-OK]i((ecn")]y
[missing-OK.>>>.

In [154]:
%sendtofile --append main.py


import ustruct
# 6FFFFE259FFFFE7F1006EE801FFFFE007804000414147FF8B000004070080000
def f():
    while True:
        write_reg(47, 1) #;  // set config bit 47
        x = read_regs(8) #;  // read all reg data into array
        time.sleep_ms(50)
        print("".join([hex(ustruct.unpack("I", y)[0])[-8:]  for y in x]))
f()  

Sent 10 lines (324 bytes) to main.py.


In [126]:
#help(SPISDA)
SPISDA.init(SPISDA.IN, SPISDA.PULL_UP)
#SPISDA.init(SPISDA.PULL_UP)


In [132]:

for i in range(100):
    write_reg(47, 1) #;  // set config bit 47
    x = read_regs(8) #;  // read all reg data into array
    #print(x)
    time.sleep(1)

1766
1664
1664
1664
1668
1664


*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 6, in <module>
KeyboardInterrupt: 


In [ ]:
%disconnect